<a href="https://colab.research.google.com/github/kundajelab/feature_interactions/blob/master/zeropadding/ComputeModelPredsForEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_simulation.simdata.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltest_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/train_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/controltrain_toanalyze_bothoff_seqs.txt.gz

!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_orig_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_taloff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_gataoff_seqs.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/shuff_toanalyze_bothoff_seqs.txt.gz

--2020-07-02 23:18:36--  https://github.com/kundajelab/feature_interactions/raw/78d4a6b/zeropadding/data/test_simulation.simdata.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kundajelab/feature_interactions/78d4a6bb7cc34b940fa2b68dba2c7fb91141a165/zeropadding/data/test_simulation.simdata.gz [following]
--2020-07-02 23:18:37--  https://raw.githubusercontent.com/kundajelab/feature_interactions/78d4a6bb7cc34b940fa2b68dba2c7fb91141a165/zeropadding/data/test_simulation.simdata.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2103069 (2.0M) [application/octet-stream]
Saving to: ‘test_simulation.simd

In [3]:
!gunzip -f *.gz

In [4]:
!md5sum *

f02081f3704a2e58fe858986738422ae  controltest_toanalyze_bothoff_seqs.txt
52f8393645eb58899d9dbc1ba59c8c39  controltest_toanalyze_gataoff_seqs.txt
72e3246fb6e7da4ea275a959b1a0f9df  controltest_toanalyze_taloff_seqs.txt
2639503d51ea1163c1008844323e00f3  controltrain_toanalyze_bothoff_seqs.txt
b8ca6db6526faa57c8b68a6aa5658d8b  controltrain_toanalyze_gataoff_seqs.txt
4f0e3b9bc8d10c3ce9a3c6a8d04c82a3  controltrain_toanalyze_taloff_seqs.txt
md5sum: sample_data: Is a directory
4f6c613269dccd3e6cb59cdf8f91b90c  shuff_toanalyze_bothoff_seqs.txt
7e0165121f5b1d24d07d576555d3c27f  shuff_toanalyze_gataoff_seqs.txt
fcd89000371697a65d62ae618eaea0e9  shuff_toanalyze_orig_seqs.txt
5be0269a68c6e5538fe80c899d9497cc  shuff_toanalyze_taloff_seqs.txt
85c3688d157254921f64904e71c0372c  test_simulation.simdata
53bc49253e6696aea04c6bba9b1f2db3  test_toanalyze_bothoff_seqs.txt
fcae5a7c5a8615351c51dae3cf3626f0  test_toanalyze_gataoff_seqs.txt
ace78751d399f68b56f09ee4c7c78920  test_toanalyze_orig_seqs.txt
17a75d63

In [5]:
!pip install simdna
import simdna
from simdna import synthetic
test_data = synthetic.read_simdata_file("test_simulation.simdata")

     |████████████████████████████████| 634kB 4.6MB/s 
  ERROR: Failed building wheel for simdna
  Running setup.py clean for simdna
Failed to build simdna
    Running setup.py install for simdna ... done


In [6]:
import numpy as np


#this is set up for 1d convolutions where examples
#have dimensions (len, num_channels) 
#the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return


def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1


def inverse_anscombe_transform(vals):
  return np.square(vals/2.0) - 3.0/8

def poissonloss_rescale(vals):
  return vals*100

def read_labels_and_oracle_notransform(filename):
  labels = (np.array([float(x.split("\t")[0]) for x in open(filename)]))
  oracle = (np.array([float(x.split("\t")[1]) for x in open(filename)]))
  return labels, oracle
  

def load_onehot_sequences(filename):
  return np.array([one_hot_encode_along_channel_axis(x.rstrip())
                   for x in open(filename)])

test_onehot_data = np.array([one_hot_encode_along_channel_axis(seq)
                              for seq in test_data.sequences])

test_toanalyze_orig_onehot =\
  load_onehot_sequences("test_toanalyze_orig_seqs.txt")
test_toanalyze_gataoff_onehot =\
  load_onehot_sequences("test_toanalyze_gataoff_seqs.txt")
test_toanalyze_taloff_onehot =\
  load_onehot_sequences("test_toanalyze_taloff_seqs.txt")
test_toanalyze_bothoff_onehot =\
  load_onehot_sequences("test_toanalyze_bothoff_seqs.txt")

controltest_toanalyze_gataoff_onehot =\
  load_onehot_sequences("controltest_toanalyze_gataoff_seqs.txt")
controltest_toanalyze_taloff_onehot =\
  load_onehot_sequences("controltest_toanalyze_taloff_seqs.txt")
controltest_toanalyze_bothoff_onehot =\
  load_onehot_sequences("controltest_toanalyze_bothoff_seqs.txt")

train_toanalyze_orig_onehot =\
  load_onehot_sequences("train_toanalyze_orig_seqs.txt")
train_toanalyze_gataoff_onehot =\
  load_onehot_sequences("train_toanalyze_gataoff_seqs.txt")
train_toanalyze_taloff_onehot =\
  load_onehot_sequences("train_toanalyze_taloff_seqs.txt")
train_toanalyze_bothoff_onehot =\
  load_onehot_sequences("train_toanalyze_bothoff_seqs.txt")

controltrain_toanalyze_gataoff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_gataoff_seqs.txt")
controltrain_toanalyze_taloff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_taloff_seqs.txt")
controltrain_toanalyze_bothoff_onehot =\
  load_onehot_sequences("controltrain_toanalyze_bothoff_seqs.txt")

shuff_toanalyze_orig_onehot =\
  load_onehot_sequences("shuff_toanalyze_orig_seqs.txt")
shuff_toanalyze_gataoff_onehot =\
  load_onehot_sequences("shuff_toanalyze_gataoff_seqs.txt")
shuff_toanalyze_taloff_onehot =\
  load_onehot_sequences("shuff_toanalyze_taloff_seqs.txt")
shuff_toanalyze_bothoff_onehot =\
  load_onehot_sequences("shuff_toanalyze_bothoff_seqs.txt")


In [8]:
!wget https://github.com/kundajelab/feature_interactions/raw/165b7f6/zeropadding/trained_models_zeropadding.tgz
!tar -xzf trained_models_zeropadding.tgz trained_models_zeropadding

--2020-07-02 23:20:13--  https://github.com/kundajelab/feature_interactions/raw/165b7f6/zeropadding/trained_models_zeropadding.tgz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kundajelab/feature_interactions/165b7f6076b848ee6d5a1da8df8daef81d59af17/zeropadding/trained_models_zeropadding.tgz [following]
--2020-07-02 23:20:13--  https://raw.githubusercontent.com/kundajelab/feature_interactions/165b7f6076b848ee6d5a1da8df8daef81d59af17/zeropadding/trained_models_zeropadding.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16551288 (16M) [application/octet-stream]
Saving to: ‘trained_models_zero

In [9]:
from collections import namedtuple
import scipy.stats


PerturbedSeqs = namedtuple("PerturbedSeqs",
  ["orig_onehot", "Aoff_onehot", "Boff_onehot", "bothoff_onehot"] )


test_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=test_toanalyze_orig_onehot,
    Aoff_onehot=test_toanalyze_gataoff_onehot,
    Boff_onehot=test_toanalyze_taloff_onehot,
    bothoff_onehot=test_toanalyze_bothoff_onehot)

controltest_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=test_toanalyze_orig_onehot,
    Aoff_onehot=controltest_toanalyze_gataoff_onehot,
    Boff_onehot=controltest_toanalyze_taloff_onehot,
    bothoff_onehot=controltest_toanalyze_bothoff_onehot)

train_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=train_toanalyze_orig_onehot,
    Aoff_onehot=train_toanalyze_gataoff_onehot,
    Boff_onehot=train_toanalyze_taloff_onehot,
    bothoff_onehot=train_toanalyze_bothoff_onehot)

controltrain_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=train_toanalyze_orig_onehot,
    Aoff_onehot=controltrain_toanalyze_gataoff_onehot,
    Boff_onehot=controltrain_toanalyze_taloff_onehot,
    bothoff_onehot=controltrain_toanalyze_bothoff_onehot)

shuff_toanalyze_perturbedseqs = PerturbedSeqs(
    orig_onehot=shuff_toanalyze_orig_onehot,
    Aoff_onehot=shuff_toanalyze_gataoff_onehot,
    Boff_onehot=shuff_toanalyze_taloff_onehot,
    bothoff_onehot=shuff_toanalyze_bothoff_onehot)


PERTURBEDSEQS_DICT = {
  "test_toanalyze_perturbedseqs": test_toanalyze_perturbedseqs,
  "controltest_toanalyze_perturbedseqs": controltest_toanalyze_perturbedseqs,
  "train_toanalyze_perturbedseqs": train_toanalyze_perturbedseqs,
  "controltrain_toanalyze_perturbedseqs": controltrain_toanalyze_perturbedseqs,
  "shuff_toanalyze_perturbedseqs": shuff_toanalyze_perturbedseqs
}


def compute_prediction(model, onehot, transform):
  return transform(np.squeeze(model.predict(onehot)))


def compute_preds_on_perturbedseqs(model, perturbedseqs, transform):
  orig_onehot_preds = compute_prediction(
      model=model, onehot=perturbedseqs.orig_onehot, transform=transform)
  Aoff_onehot_preds = compute_prediction(
      model=model, onehot=perturbedseqs.Aoff_onehot, transform=transform)
  Boff_onehot_preds = compute_prediction(model=model,
                      onehot=perturbedseqs.Boff_onehot, transform=transform)
  bothoff_onehot_preds = compute_prediction(model=model,
                          onehot=perturbedseqs.bothoff_onehot,
                          transform=transform)
  return {"orig_onehot_preds": orig_onehot_preds,
          "Aoff_onehot_preds": Aoff_onehot_preds,
          "Boff_onehot_preds": Boff_onehot_preds,
          "bothoff_onehot_preds": bothoff_onehot_preds}


def compute_preds(model, transform):
  to_return = {}
  for key, perturbedseqs in PERTURBEDSEQS_DICT.items():
    to_return[key] = compute_preds_on_perturbedseqs(
                      model=model, perturbedseqs=perturbedseqs,
                      transform=transform)
  return to_return
  

In [ ]:
import glob
import keras
import h5py


class RescalePoissonLoss(object):

  def __init__(self, n):
    self.predrescale_factor = n
    self.name = "RescalePoisson_"+str(self.predrescale_factor)

  def __call__(self, y_true, y_pred, *args, **kwargs):
    return keras.losses.poisson(y_true, y_pred*self.predrescale_factor)
  
  def get_config(self):
        return {"n": self.predrescale_factor}


class RescaleMSE(object):
  
  def __init__(self, n):
    self.predrescale_factor = n
    self.__name__ = "RescaleMSE_"+str(self.predrescale_factor)

  def __call__(self, y_true, y_pred):
    return keras.metrics.mse(y_true=y_true, y_pred=y_pred*self.predrescale_factor)
  
  def get_config(self):
        return {"n": self.predrescale_factor}


def get_transform(modelfile):
  if "anscombe" in modelfile:
    return inverse_anscombe_transform
  elif "poisson" in modelfile:
    return poissonloss_rescale
  else:
    raise RuntimeError("What is the transform for",modelfile,"?")

model_files = glob.glob("trained_models_zeropadding/*.h5")
!mkdir model_preds_zeropadding

for model_file in model_files:
  #some hacky stuff in place to make model loading of the custom loss/metrics
  # actually works...this doesn't actually matter at this stage because we
  # are not retraining the model.
  with keras.utils.CustomObjectScope(
      {'RescalePoissonLoss': RescalePoissonLoss,
       'RescaleMSE': RescaleMSE,
       "{'class_name': 'RescaleMSE', 'config': {'n': 100}}": lambda: RescaleMSE(100) }):
    model = keras.models.load_model(model_file)
  transform = get_transform(model_file)
  model_preds = compute_preds(model=model, transform=transform)
  all_test_preds = compute_prediction(model=model,
                                      onehot=test_onehot_data,
                                      transform=transform)
  preds_savefile = "model_preds_zeropadding/"+(model_file.split("/")[-1])
  !rm $preds_savefile
  print("File",preds_savefile)
  fh = h5py.File(preds_savefile, "w")
  fh.create_dataset("all_test_preds", data=all_test_preds)
  for key in model_preds:
    grp = fh.create_group(key)
    for subkey in model_preds[key]:
      grp.create_dataset(subkey, data=model_preds[key][subkey])
  fh.close()

rm: cannot remove 'model_preds_zeropadding/negcontrol_model-rescalepoisson_skipconn_l1reg-1e-05_seed400_onwci.h5': No such file or directory
File model_preds_zeropadding/negcontrol_model-rescalepoisson_skipconn_l1reg-1e-05_seed400_onwci.h5
rm: cannot remove 'model_preds_zeropadding/poscontrol_model-anscombemse_arch2_l1reg-0.0001_seed100_kqujx.h5': No such file or directory
File model_preds_zeropadding/poscontrol_model-anscombemse_arch2_l1reg-0.0001_seed100_kqujx.h5
rm: cannot remove 'model_preds_zeropadding/poscontrol_model-rescalepoisson_skipconn_l1reg-1e-05_seed500_nbmah.h5': No such file or directory
File model_preds_zeropadding/poscontrol_model-rescalepoisson_skipconn_l1reg-1e-05_seed500_nbmah.h5
rm: cannot remove 'model_preds_zeropadding/negcontrol_model-rescalepoisson_sanitycheck_l1reg-0.0001_seed500_gpagt.h5': No such file or directory
File model_preds_zeropadding/negcontrol_model-rescalepoisson_sanitycheck_l1reg-0.0001_seed500_gpagt.h5
rm: cannot remove 'model_preds_zeropadding

In [ ]:
from google.colab import files
!tar -czf model_preds_zeropadding.tgz model_preds_zeropadding
files.download("model_preds_zeropadding.tgz")